In [17]:
import networkx as nx
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram
from functools import reduce

import numpy as np
from scipy.optimize import minimize

In [18]:
cst = [-100, -1, -20]

In [19]:
depth = 5
params = np.ones(depth * 2)

In [20]:
GateX = np.array([[0, 1],[1, 0]])
GateY = np.array([[0, -1j],[1j, 0]])
GateZ = np.array([[1, 0],[0, -1]])

# 定义σ+和σ-矩阵
sigma_plus = np.array([[0, 1], [0, 0]])
sigma_minus = np.array([[0, 0], [1, 0]])

def add_in_target(num_qubits, target_qubit, gate=np.array([[1, 0],[0, -1]])):
    H = np.eye(2 ** (target_qubit))
    H = np.kron(H, gate)
    H = np.kron(H, np.eye(2 ** (num_qubits - 1 - target_qubit)))
    return H

def calculate_hamiltonian(v, w):
    n = len(v[0])
    m = len(v)
    hamiltonian = np.zeros((2**n, 2**n))

    for i in range(m):
        term1 = reduce(np.kron, [np.linalg.matrix_power(sigma_plus, v[i][j]) for j in range(n)])
        term2 = reduce(np.kron, [np.linalg.matrix_power(sigma_minus, w[i][j]) for j in range(n)])
        term3 = reduce(np.kron, [np.linalg.matrix_power(sigma_plus, w[i][j]) for j in range(n)])
        term4 = reduce(np.kron, [np.linalg.matrix_power(sigma_minus, v[i][j]) for j in range(n)])

        hamiltonian += term1 @ term2 + term3 @ term4

    return hamiltonian

def first_nonzero_index(arr, total_bits=3):
    for i, num in enumerate(arr):
        if num != 0:
            binary_repr = format(i, '0' + str(total_bits) + 'b')
            return binary_repr
        
def gnrt_Hd():
  Hd = np.zeros((2**3, 2**3)).astype(np.complex128)
  Hd += add_in_target(3, 0, GateZ)
  Hd += add_in_target(3, 1, GateZ)
  Hd += add_in_target(3, 2, GateZ)
  return Hd

In [21]:
# dtype=np.complex128
def add_in_target(num_qubits, target_qubit, gate=np.array([[1, 0],[0, -1]])):
    H = np.eye(2 ** (target_qubit))
    H = np.kron(H, gate)
    H = np.kron(H, np.eye(2 ** (num_qubits - 1 - target_qubit)))
    return H


def generate_Hp():
    Hp = np.zeros((2**3, 2**3))
    for i in range(3):
        Hp +=  cst[i] * (add_in_target(3, i) - np.eye(2**3)) / 2
    return Hp
Hp = generate_Hp()

In [22]:
def gnrt_Hd():
  Hd = np.zeros((2**3, 2**3)).astype(np.complex128)
  Hd += (add_in_target(3, 0, GateX) @ add_in_target(3, 1, GateX) + add_in_target(3, 0, GateY) @ add_in_target(3, 1, GateY))
  Hd += (add_in_target(3, 1, GateX) @ add_in_target(3, 2, GateX) - add_in_target(3, 1, GateY) @ add_in_target(3, 2, GateY))
  Hd += (add_in_target(3, 2, GateX) @ add_in_target(3, 0, GateX) - add_in_target(3, 2, GateY) @ add_in_target(3, 0, GateY))
  # Hd += add_in_target(1, 0, GateX)
  # Hd += add_in_target(3, 1, GateX)
  # Hd += add_in_target(3, 2, GateX)
  return Hd

In [23]:
v = np.array([[1, 0, 0],
              [0, 1, 0],
              [0, 0, 1]])
w = np.array([[0, 1, 0], 
              [0, 0, 1],
              [1, 0, 0]])
# Hd = gnrt_Hd()
Hd = calculate_hamiltonian(v, w)

In [24]:
# 求解本征值和本征态
eigenvalues, eigenvectors = np.linalg.eig(Hd)
print(eigenvalues)
# 输出结果
for i in range(len(eigenvalues)):
  print("\nEigenvalues:")
  print(f'{eigenvalues[i].real:.4f}')
  print("Eigenvectors:")
  print([f'{num:.4f}' for num in eigenvectors[:, i].real])

[-1.  2. -1.  2. -1. -1.  0.  0.]

Eigenvalues:
-1.0000
Eigenvectors:
['0.0000', '0.0000', '0.0000', '0.4082', '0.0000', '0.4082', '-0.8165', '0.0000']

Eigenvalues:
2.0000
Eigenvectors:
['0.0000', '-0.0000', '0.0000', '0.5774', '0.0000', '0.5774', '0.5774', '0.0000']

Eigenvalues:
-1.0000
Eigenvectors:
['0.0000', '0.7682', '-0.3841', '-0.1384', '-0.3841', '-0.1384', '0.2768', '0.0000']

Eigenvalues:
2.0000
Eigenvectors:
['0.0000', '-0.5485', '-0.5485', '-0.1803', '-0.5485', '-0.1803', '-0.1803', '0.0000']

Eigenvalues:
-1.0000
Eigenvectors:
['0.0000', '0.4153', '0.3649', '-0.1835', '-0.7802', '-0.0410', '0.2244', '0.0000']

Eigenvalues:
-1.0000
Eigenvectors:
['0.0000', '-0.2422', '0.0380', '-0.6350', '0.2042', '0.7005', '-0.0654', '0.0000']

Eigenvalues:
0.0000
Eigenvectors:
['1.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000']

Eigenvalues:
0.0000
Eigenvectors:
['0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '0.0000', '1.0000']


In [25]:
from scipy.linalg import expm
def build_circ(params):
  qc = QuantumCircuit(3)
  beta = params[:depth]
  gamma = params[depth:]
  qc.initialize(eigenvectors[:, 1].real, range(3))
  # qc.x(0)
  # qc.x(2)
  for dp in range(depth):
    qc.unitary(expm(-1j * gamma[dp] * Hp), range(3)) # transpile
    qc.unitary(expm(-1j * beta[dp] * Hd), range(3))
  qc.measure_all()
  return qc

In [26]:
# print(build_circ(params))

In [27]:
def cost_function(x):
  num = [int(char) for char in x]
  C = 0
  for i in range(3):
    C += cst[i] * num[i]
  return C

def compute_expectation(counts):
  EV = 0
  total_count = 0
  for x, count in counts.items():
    C = cost_function(x)
    EV += C*count
    total_count += count

  return EV/total_count


def expectation_from_sample(shots = 2000):
  backend = Aer.get_backend('qasm_simulator')
  backend.shots = shots

  def execute_circ(theta):
    qc = build_circ(theta)
    counts = backend.run(qc, seed_simulator=10, shots=shots).result().get_counts()
    return compute_expectation(counts)
  
  return execute_circ

In [28]:
from numpy.lib.utils import source
from scipy.optimize import minimize
import numpy as np
# 初始化迭代计数器
iteration_count = 0
def test(pen, dep, par):
  global penalty, depth, params, iteration_count
  iteration_count = 0
  penalty = pen
  depth = dep
  params = par
  expectation = expectation_from_sample()
  def callback(x):
      global iteration_count
      iteration_count += 1
      if iteration_count % 10 == 0:
          print(f"Iteration {iteration_count}, Result: {expectation(x)}")
  # 设定最大迭代次数
  max_iterations = 1000

  # 使用 COBYLA 方法进行最小化，并设置 callback 函数
  res = minimize(expectation, params, method='COBYLA', options={'maxiter': max_iterations}, callback=callback)
  # 输出最终结果
  print("Final Result:", res)
  backend = Aer.get_backend('aer_simulator')
  backend.shots = 100000

  shots=100000
  qc_res = build_circ(params=res.x)

  counts = backend.run(qc_res, seed_simulator=10, shots = shots).result().get_counts()
  # plot_histogram(counts)
  sorted_counts = sorted(counts, key=counts.get, reverse=True)
  print("\n----------------- Full result ---------------------")
  print("selection\t\tprobability\tvalue")
  print("---------------------------------------------------")
  for x in sorted_counts[:20]:
    print(x, "{:.1f}%".format(counts[x] / shots * 100), cost_function(x))

In [29]:
test(40, 5, np.full(5 * 2, np.pi/5))

Iteration 10, Result: -61.916


Iteration 20, Result: -47.9585
Iteration 30, Result: -86.739
Iteration 40, Result: -82.3335
Iteration 50, Result: -107.566
Iteration 60, Result: -111.119
Iteration 70, Result: -112.096
Iteration 80, Result: -113.1515
Iteration 90, Result: -114.934
Iteration 100, Result: -113.752
Iteration 110, Result: -116.3065
Iteration 120, Result: -117.925
Iteration 130, Result: -118.4665
Iteration 140, Result: -118.5805
Iteration 150, Result: -118.5805
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: -118.4815
       x: [ 5.305e-01  4.075e-01  6.190e-01  1.116e+00  1.629e+00
            1.682e+00  6.203e-01  5.858e-01  1.663e+00  5.080e-01]
    nfev: 158
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
101 93.3% -120
110 6.5% -101
011 0.2% -21


In [30]:
test(40, 3, np.full(3 * 2, np.pi/3))

Iteration 10, Result: -80.5245
Iteration 20, Result: -91.278
Iteration 30, Result: -108.3555
Iteration 40, Result: -112.7935
Iteration 50, Result: -114.341
Iteration 60, Result: -114.6045
Iteration 70, Result: -114.637
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: -114.677
       x: [ 7.611e-01  2.002e+00  1.013e+00  1.059e+00  1.046e+00
            1.048e+00]
    nfev: 78
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
101 79.0% -120
110 19.1% -101
011 1.9% -21


In [31]:
# for i in range(1, 20):
#   print(f'depth == {i}')
#   test(20, i, np.ones(2 * i))